# Project 2- ME5406


#### Sheth Riya Nimish A0176880R
#### Srishti Ganguly A0170868R

# Robot Hand Playing the Piano

### Introduction

This is the code for the environment of the PianoHand. The objective of the agent is to play the key of the piano stated by the user. In this example, the user can specify either of the four keys C, D, E or F.
For this, the agent needs to learn which finger to move and by how much to move to play the key specified by the user.

<img src="files/Pictures/finger.png">

Each finger as seen from the diagram above has two links which can be moved be angles $\theta_1$ and $\theta_2$. In this environment, the range of movement for $\theta_1$ and $\theta_2$ is between 30-60 degrees both inclusive. To replicate real life models, the fingers of the hand have been given different lengths.

The action space of this environment is discrete and is of size 16. Each finger has 4 different actions: increase  by $\theta_1$ by 1, decrease $\theta_1$  by 1, increase $\theta_2$  by 1 and decrease $\theta_2$  by 1.
The state space of this environment is a box space with discrete values. It is an 8-dimensional state space with each dimension having a lower bound of 30 and upper bound of 60.

The agent gets a reward of +200 if it plays the desired key and the episode ends. It gets a penalty of -25 if it plays the wrong key and the episode ends and a penalty of -10 if it tries to exceed the angle bounds of 30-60 degrees and the episode ends.

### Mathematical Computation of the Coordinates

Each finger is placed at a distance of 50 apart in the Z dimension. Hence, the Z coordinate for all the fingers remains unchanged throughout. That is the z-coordinate of finger1 is 0, of finger2 is 50, finger3 is 150 and so on.

Calculations for finger1:

Point a as seen in the above diagram is defined as the origin having coordinates (0, 0, 0).
Point b can be calculated using $\theta_1$ and length of the finger. 
l = length of finger's two joints

$$[b(x),\hspace{2mm}  b(y), \hspace{2mm}b(z)]= [l * sin(\theta_1),    \hspace{4mm}  l *cos(\theta_1),   \hspace{4mm}   0]$$

Point c can be calculated using \theta_2 and Point b

$$[c(x), \hspace{2mm}c(y), \hspace{2mm}c(z)]= [-b(x)*cos \theta_2  -  b(y)* sin\theta_2  + b(x),      \hspace{4mm}    -b(x)*sin \theta_2  -  b(y)* cos\theta_2  + b(y),   \hspace{4mm}     0]$$

The calculations for the remaining fingers are identical to the above only the Z-dimension changes. The formula for the Z-coordinate is $i*50$ where i is the finger number.

### Visualizing

The render functions show the top view and the side view of the fingers. We have tried to replicate the following views for better demonstration of our model.
$$Side \hspace{2mm} View$$
<img src="files/Pictures/PianoSide.png" style="width: 150px;"/>
$$Top \hspace{2mm} View$$
<img src="files/Pictures/PianoTop.png" style="width: 100px;"/> 

Source: https://www.labmanager.com/news/3-d-printed-robot-hand-plays-the-piano-2878  


### An example gif of when it plays the right key (around timestep 500):
<img src="GIFS/examplerightkey.gif" width="750" align="center">

### An example gif of when it plays the wrong key (around timestep 170):
<img src="GIFS/examplewrongkey.gif" width="750" align="center">

For inceased clarity of the gifs, the gifs have been uploaded in the zip folder as well.

#### Importing the required libraries

In [ ]:
import numpy as np
import gym
from gym import spaces
import random
import math
from PIL import Image, ImageDraw, ImageFont
import os
import time
from collections import deque
from time import sleep
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import imageio
import datetime

#### Creating the class for the environment

We have made use of OpenAi's gym package to create our Custom Environment

In [ ]:
class PianoHandEnv(gym.Env):
    
    #initialization of class variables
    number_of_fingers=4
    
    #the lengths of the fingers
    finger_length= [100, 120, 130, 140]
    
    #the configurations of the 8 angles
    finger_currangle= [(0, 0), (0, 0), (0, 0), (0, 0)] 
    
    #the coordinates of the link positions for the four fingers
    finger_link_position=[(0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0)]
    
    #the coordinates of the finger tip positions for the four fingers
    finger_final_position=[(0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0)] 
    
    #the initial positions of all the angles        
    finger_initialtheta= [(45, 45), (45, 45), (45, 45), (45, 45)]
    
    #to store the goal or the key to be played as stated by the user
    goalkeep=''
    
    #the integer value of the key to be played
    goalnum=0  
    
    #counter to keep track of whether the goal has been achieved
    goalachieved=0
    
    key_names=['C', 'D', 'E', 'F']
    
    #the positions of the keyboard were defined manually and were mathematically proven to be reachable from all angle positions
    key_pos= [(80, -85, 0), (80, -85, 50), (80, -85, 100), (80, -85, 150)]
    key_length= 30
    key_breadth= 100
    
    metadata = {'render.modes': ['human', 'rgb_array']} 
    
    
    def __init__(self, goal):
        super(PianoHandEnv, self).__init__() 
        self.action_space = spaces.Discrete(16) 
        self.observation_space = spaces.Box(np.array([30, 30, 30, 30, 30, 30, 30, 30]), np.array([60, 60, 60, 60, 60, 60, 60, 60])) 
        self.goalkeep= goal
        
        #storing the integer value of the goal
        if(goal=='C'):
            self.goalnum=3
        elif(goal=='D'):
            self.goalnum= 2
        elif(goal=='E'):
            self.goalnum= 1
        elif(goal=='F'):
            self.goalnum= 0
    
        self.finger_currangle= self.finger_initialtheta
        
        #defining the link position and the tip positions of the fingers based on the initial angles
        for i in range(self.number_of_fingers):
       
            th1= math.radians(self.finger_currangle[i][0])
            th2= math.radians(self.finger_currangle[i][1])
      
            self.finger_link_position[i]=(self.finger_length[i]*np.sin(th1), self.finger_length[i]*np.cos(th1), i*50)
            self.finger_final_position[i]= ((-self.finger_link_position[i][0])*np.cos(th2)-(-self.finger_link_position[i][1])*np.sin(th2)+self.finger_link_position[i][0], (-self.finger_link_position[i][0])*np.sin(th2)+(-self.finger_link_position[i][1])*np.cos(th2)+self.finger_link_position[i][1], i*50)
         

    def step(self, action):
        
        done = False
        reward = 0
        
        #checking if the angles of the first finger will still be within bounds after executing the given action
        if(self.finger_currangle[0][0]>=60 and action==0):
            self.goalachieved=-1
            return self.finger_currangle,-10, True, self.finger_link_position, self.finger_final_position
        
        if(self.finger_currangle[0][0]<=30 and action==1):
            self.goalachieved=-1
            return self.finger_currangle, -10, True, self.finger_link_position, self.finger_final_position
        
        if(self.finger_currangle[0][1]>= 60 and action ==2):
            self.goalachieved=-1
            return self.finger_currangle, -10, True, self.finger_link_position, self.finger_final_position
        
        if(self.finger_currangle[0][1]<= 30 and action ==3):
            self.goalachieved=-1
            return self.finger_currangle, -10, True, self.finger_link_position, self.finger_final_position
   
        #checking if the angles of the second finger will still be within bounds after executing the given action
        if(self.finger_currangle[1][0]>= 60 and action==4):
            self.goalachieved=-1
            return self.finger_currangle, -10, True, self.finger_link_position, self.finger_final_position
        
        if(self.finger_currangle[1][0]<= 30 and action==5):
            self.goalachieved=-1
            return self.finger_currangle, -10, True, self.finger_link_position, self.finger_final_position
        
        if(self.finger_currangle[1][1]>= 60 and action ==6):
            self.goalachieved=-1
            return self.finger_currangle, -10, True, self.finger_link_position, self.finger_final_position
        
        if(self.finger_currangle[1][1]<= 30 and action ==7):
            self.goalachieved=-1
            return self.finger_currangle, -10, True, self.finger_link_position, self.finger_final_position
     
        #checking if the angles of the third finger will still be within bounds after executing the given action
        if(self.finger_currangle[2][0]>= 60 and action==8):
            self.goalachieved=-1
            return self.finger_currangle, -10, True, self.finger_link_position, self.finger_final_position
        
        if(self.finger_currangle[2][0]<= 30 and action==9):
            self.goalachieved=-1
            return self.finger_currangle, -10, True, self.finger_link_position, self.finger_final_position
        
        if(self.finger_currangle[2][1]>= 60 and action ==10):
            self.goalachieved=-1
            return self.finger_currangle, -10, True, self.finger_link_position, self.finger_final_position
        
        if(self.finger_currangle[2][1]<= 30 and action ==11):
            self.goalachieved=-1
            return self.finger_currangle, -10, True, self.finger_link_position, self.finger_final_position
    
        #checking if the angles of the third finger will still be within bounds after executing the given action
        if(self.finger_currangle[3][0]>= 60 and action==12):
            self.goalachieved=-1
            return self.finger_currangle, -10, True, self.finger_link_position, self.finger_final_position
        
        if(self.finger_currangle[3][0]<= 30 and action==13):
            self.goalachieved=-1
            return self.finger_currangle, -10, True, self.finger_link_position, self.finger_final_position
        
        if(self.finger_currangle[3][1]>= 60 and action ==14):
            self.goalachieved=-1
            return self.finger_currangle, -10, True, self.finger_link_position, self.finger_final_position
        
        if(self.finger_currangle[3][1]<= 30 and action ==15):
            self.goalachieved=-1
            return self.finger_currangle, -10, True, self.finger_link_position, self.finger_final_position

        angle_change = 1
        
        """
        Changing the angle_change will change the magnitude of the action taken.
        For instance changing it to 5, will make each action increase or decrease theta by 5
        """
        
        #changing the thetas according to the action given
        
        #first finger
        if action ==0:
            self.finger_currangle[0]= (self.finger_currangle[0][0]+angle_change, self.finger_currangle[0][1])
       
        elif action ==1:
            self.finger_currangle[0]= (self.finger_currangle[0][0]-angle_change, self.finger_currangle[0][1])
            
        elif action ==2:
            self.finger_currangle[0]= (self.finger_currangle[0][0], self.finger_currangle[0][1]+angle_change)
            
        elif action ==3:
            self.finger_currangle[0]= (self.finger_currangle[0][0], self.finger_currangle[0][1]-angle_change)
        
        #second finger
        elif action ==4:
            self.finger_currangle[1]= (self.finger_currangle[1][0]+angle_change, self.finger_currangle[1][1])
       
        elif action ==5:
            self.finger_currangle[1]= (self.finger_currangle[1][0]-angle_change, self.finger_currangle[1][1])
            
        elif action ==6:
            self.finger_currangle[1]= (self.finger_currangle[1][0], self.finger_currangle[1][1]+angle_change)
            
        elif action ==7:
            self.finger_currangle[1]= (self.finger_currangle[1][0], self.finger_currangle[1][1]-angle_change)
        
        #third finger
        elif action ==8:
            self.finger_currangle[2]= (self.finger_currangle[2][0]+angle_change, self.finger_currangle[2][1])
       
        elif action ==9:
            self.finger_currangle[2]= (self.finger_currangle[2][0]-angle_change, self.finger_currangle[2][1])
            
        elif action ==10:
            self.finger_currangle[2]= (self.finger_currangle[2][0], self.finger_currangle[2][1]+angle_change)
            
        elif action ==11:
            self.finger_currangle[2]= (self.finger_currangle[2][0], self.finger_currangle[2][1]-angle_change)
               
        #fourth finger
        elif action ==12:
            self.finger_currangle[3]= (self.finger_currangle[3][0]+angle_change, self.finger_currangle[3][1])
       
        elif action ==13:
            self.finger_currangle[3]= (self.finger_currangle[3][0]-angle_change, self.finger_currangle[3][1])
            
        elif action ==14:
            self.finger_currangle[3]= (self.finger_currangle[3][0], self.finger_currangle[3][1]+angle_change)
            
        elif action ==15:
            self.finger_currangle[3]= (self.finger_currangle[3][0], self.finger_currangle[3][1]-angle_change)
            
        
        #to calculate the coordinates of the link and the final position of each finger
        for i in range(self.number_of_fingers):
            
            th1= math.radians(self.finger_currangle[i][0])
            th2= math.radians(self.finger_currangle[i][1])
            l1= self.finger_length[i]
            
            link= (l1*np.sin(th1), l1*np.cos(th1), i*50)
            final= ((-link[0])*np.cos(th2)-(-link[1])*np.sin(th2)+link[0], (-link[0])*np.sin(th2)+(-link[1])*np.cos(th2)+link[1], i*50)

            self.finger_link_position[i]= link
            self.finger_final_position[i]= final
           
            """
            A pianokey is played if the tip of the finger is in the 3D rectangular space of that key.
              
            To check whether the key is played, we check whether the 
            x coordinate of the tip of the finger(stored in coordinates final)
            is between the x ranges of the goal-pianokey, the ycoordinates are between
            the y ranges of the goal-pianokey and the zcoordinates is equal to the 
            z coordinate of the goal-pianokey.
        
            """
            
            if (final[0] > self.key_pos[i][0] and final[0] < self.key_pos[i][0] + self.key_breadth):
                
                if (final[1]> self.key_pos[i][1] and final[1] < self.key_pos[i][1] + self.key_length):
                    
                    if(final[2]== self.key_pos[i][2]):
                        
                        if i== self.goalnum:
                            
                            reward=200
                            done = True
                            self.goalachieved= self.number_of_fingers+1
                            #print("Played Key!")
                            break
                            
                        else:
                            
                            reward=-25
                            self.goalachieved= i
                            done= True
                            #print("Played Wrong Key!")
                            
            
        return self.finger_currangle, reward, done, self.finger_link_position, self.finger_final_position
    

    # function to reset all the parameters to the original ones at the end of the episode
    def reset(self):
        
        self.finger_initialtheta= [(45, 45), (45, 45), (45, 45), (45, 45)]
        """
        Code for exploring starts and randomizing inital angles 

        for i in range(4):
            finger_initialtheta[i][0]= random.randit(30, 60)
            finger_initialtheta[i][1]= random.randit(30, 60)
        """

        self.finger_currangle= self.finger_initialtheta
        
        #defining the link position and the tip positions of the fingers based on the initial angles
        for i in range(self.number_of_fingers):
       
            th1= math.radians(self.finger_currangle[i][0])
            th2= math.radians(self.finger_currangle[i][1])
      
            self.finger_link_position[i]=(self.finger_length[i]*np.sin(th1), self.finger_length[i]*np.cos(th1), i*50)
            self.finger_final_position[i]= ((-self.finger_link_position[i][0])*np.cos(th2)-(-self.finger_link_position[i][1])*np.sin(th2)+self.finger_link_position[i][0], (-self.finger_link_position[i][0])*np.sin(th2)+(-self.finger_link_position[i][1])*np.cos(th2)+self.finger_link_position[i][1], i*50)
        
        return self.finger_currangle
    

    def render(self, d, done):
 
        #Creating an episode frame for visualizing
        img= Image.new("RGB", (1000, 1000))
        img1 = ImageDraw.Draw(img)
        pixels= img.load()
        draw = ImageDraw.Draw(img)
        myfont = ImageFont.truetype('Fonts/Oswald/Oswald-VariableFont_wght.ttf', 25)
        
        #printing the links and joints of the finger for the side view
        for i in range(self.number_of_fingers):
            
            link= self.finger_link_position[i]
            final= self.finger_final_position[i]
            
            #transposing the origin so that negative coordinates can be acounted for
            origin_pos_x = 250
            origin_pos_y = 250
        
            if(link[0]>0):
                link1_pos_x = link[0] + 250
            else:
                link1_pos_x = 250 + link[0]
        
            if(final[0]>0):
                final_pos_x = final[0] + 250
            else:
                final_pos_x = 250+ final[0]
        
            if(link[1]>0):
                link1_pos_y = 250 - link[1]
            else:
                link1_pos_y = -link[1] + 250
        
            if(final[1]>0):
                final_pos_y = 250 - final[1]
            else:
                final_pos_y = -final[1] + 250
                
            """
            Here the nested for loops help the joint 
            to be more noticable rather than just a point
            """
            
            #image of the robot link 1   
            for i in range (5):
                for j in range (5):
                    pixels[link1_pos_x+i, link1_pos_y+j]= (255, 0, 0)
                    
            #image for tip of finger
            for i in range (5):
                for j in range (5):
                    pixels[final_pos_x+i, final_pos_y+j]= (128, 0, 128, 255)
        
            #image for origin 
            for i in range (5):
                for j in range (5):
                    pixels[250+i, 250+j]= (255, 255, 0, 255)
        
                
        
            img1.line([(origin_pos_x, origin_pos_y), (link1_pos_x, link1_pos_y)], fill ="blue", width = 3) 
            img1.line([(link1_pos_x, link1_pos_y), (final_pos_x, final_pos_y)], fill ="blue", width = 3)
        
        
        #print the white key for the side view
        for i in range(self.key_breadth):
            for j in range(self.key_length):
                pixels[self.key_pos[0][0]+i+250, -self.key_pos[0][1]-j+250]= (255, 255, 255)
        
        
                    
                    
        #prints the white keys for the top view
        for k in range(4):
            for i in range(self.key_breadth+100):
                for j in range(50):
                    pixels[self.key_pos[k][0]+i+550, -self.key_pos[k][2]-j+325]= (255, 255, 255)
                               
                    
        """
        Plays a concluding message in the gif:
        according to the outcome of the episode.
        
        """
                    
        if (done==True):
            if(self.goalachieved==5):
                draw.text((450, 700), "PLAYED KEY!", (255, 255, 255), font=myfont)
                for i in range(self.key_breadth+100):
                    for j in range(50):
                        pixels[self.key_pos[self.goalnum][0]+i+550, -self.key_pos[self.goalnum][2]-j+325]= (0, 255, 0, 255)
            
            elif(self.goalachieved==-1):
                draw.text((450, 700), "Angle gone out of bound!", (255, 255, 255), font=myfont)
            
            else:
                draw.text((450, 700), "PLAYED WRONG KEY!", (255, 255, 255), font=myfont)
                for i in range(self.key_breadth+100):
                    for j in range(50):
                        print(self.goalachieved)
                        pixels[self.key_pos[self.goalachieved][0]+i+550, -self.key_pos[self.goalachieved][2]-j+325]= (255, 0, 0)
        
        
                    
        #prints black lines to separate the white keys in the top view
        for k in range(4):
            for i in range(self.key_breadth+100):
                pixels[self.key_pos[k][0]+i+550, -self.key_pos[k][2]+325]= (0, 0, 0, 255)
                
        
        #lengths of the black keys just for visualisation purposes
        black_key_breadth= 25
        black_key_length = 100
        black_key_start= 180
        
        
        #prints black keys in the top view
        for k in range(3):
            for i in range(black_key_length):
                for j in range (25):
                    pixels[180+i+550, -37.5-50*k-j +325]= (0, 0, 0, 255)
                    
                    
        #printing the links and joints of the finger for the top view
        for k in range(self.number_of_fingers):
            
            #prints the longer horizontal line, between origin and link1 or origin and finalposition
            if(self.finger_final_position[k][0]>self.finger_link_position[k][0]):
                temp= self.finger_final_position[k][0]
            else:
                temp= self.finger_link_position[k][0]
            
            img1.line([(550, -k*50+300), (temp+550, -k*50+300)], fill ="blue", width = 3) 
            
            
            for i in range (5):
                for j in range (5):
                    pixels[self.finger_final_position[k][0]+i+550, -k*50+j-2.5+300]= (255, 0, 0)
                    
            for i in range (5):
                for j in range (5):
                    pixels[i+550, -k*50+j-2.5+300]= (255, 255, 0, 255) 
            
            for i in range (5):
                for j in range (5):
                    pixels[self.finger_link_position[k][0]+i+550, -k*50+j-2.5+300]= (128, 0, 128, 255)
        
        
        
        draw.text((0, 0), "PANIC IN THE DISCO", (255, 255, 255), font=myfont)
        
        draw.text((270, 80), "Side View", (128,  0, 128, 255), font=myfont)
        draw.text((650, 80), "Top View", (128,  0, 128, 255), font=myfont)
        
        draw.text((850, 130), "C", (255, 255, 255), font=myfont)
        draw.text((850, 180), "D", (255, 255, 255), font=myfont)
        draw.text((850, 230), "E", (255, 255, 255), font=myfont)
        draw.text((850, 280), "F", (255, 255, 255), font=myfont)
        
        draw.text((0, 400), "Finger 1: Theta1: %d"%self.finger_currangle[0][0], (255, 255, 255), font=myfont)
        draw.text((0, 425), "Finger 1: Theta2: %d"%self.finger_currangle[0][1], (255, 255, 255), font=myfont)
        draw.text((0, 475), "Finger 2: Theta1: %d"%self.finger_currangle[1][0], (255, 255, 255), font=myfont)
        draw.text((0, 500), "Finger 2: Theta2: %d"%self.finger_currangle[1][1], (255, 255, 255), font=myfont)
        draw.text((0, 550), "Finger 3: Theta1: %d"%self.finger_currangle[2][0], (255, 255, 255), font=myfont)
        draw.text((0, 575), "Finger 3: Theta2: %d"%self.finger_currangle[2][1], (255, 255, 255), font=myfont)
        draw.text((0, 625), "Finger 4: Theta1: %d"%self.finger_currangle[3][0], (255, 255, 255), font=myfont)
        draw.text((0, 650), "Finger 4: Theta2: %d"%self.finger_currangle[3][1], (255, 255, 255), font=myfont)
        
        
        draw.text((0, 700), "Goal: "+ self.goalkeep, (255, 255, 255), font=myfont)
        draw.text((0, 800), "Time-step: %d"%d, (255, 255, 255), font=myfont)
        
                  
        #uncomment this if need to see or save the image
        #img.show()
        #img= img.save("ZFrames_2_%d.png"%d)
        
        
        return img
    
#END OF ENVIRONMENT